In [27]:
import os
import datetime
import pandas as pd
import xgboost as xgb

gyms = ['Munich East', 'Munich West', 'Munich South', 'Dortmund', 'Frankfurt', 'Regensburg']
gyms_dict = {'Munich East': 'muenchen-ost', 'Munich West': 'muenchen-west', 'Munich South': 'muenchen-sued', 
            'Dortmund': 'dortmund', 'Frankfurt': 'frankfurt', 'Regensburg': 'regensburg'}

In [22]:
boulderdf = pd.read_csv("boulderdata.csv")

In [24]:
latesttime = boulderdf.iloc[0].current_time
for i in range(len(gyms)):
    row = boulderdf.iloc[i]
    if row.current_time != latesttime:
        break
latestdata = boulderdf.head(i)
latestdata

,current_time,gym_name,occupancy,waiting,weather_temp,weather_status
0,2021/06/17 16:30,muenchen-ost,31.0,0.0,30.98,Clear
1,2021/06/17 16:30,muenchen-west,23.0,0.0,30.98,Clear
2,2021/06/17 16:30,muenchen-sued,11.0,0.0,30.98,Clear
3,2021/06/17 16:30,dortmund,3.0,0.0,34.44,Clear
4,2021/06/17 16:30,regensburg,23.0,0.0,31.34,Clear


In [19]:
model2 = xgb.Booster()
model2.load_model('xgb.model')

In [25]:
def preprocess(df: pd.DataFrame) -> (np.array, np.array):

    # datetime processing. obtain weekday, and time as hours and minutes rounded up
    # ex. 16:45 becomes 16.75. 15:30 becomes 15.50
    df['current_time'] = pd.to_datetime(df['current_time'])
    df['weekday'] = df['current_time'].apply(lambda x: x.strftime('%A'))
    df['time'] = df['current_time'].apply(lambda x: x.hour + x.minute/60)
    df.drop('current_time', axis=1, inplace=True)

    # join occupancy and waiting into a single column
    df['occupancy'] = df.apply(lambda r: r.occupancy + r.waiting/10, axis=1)
    df.drop('waiting', axis=1, inplace=True)

    # one-hot encode the categorical variables
    df = one_hot_encode_col(df, 'gym_name')
    df = one_hot_encode_col(df, 'weekday')
    df = one_hot_encode_col(df, 'weather_status')

    # extract X,y from df
    y = df['occupancy'].to_numpy()
    X = df.drop('occupancy', axis=1).to_numpy()

    return X, y

test_X, _ = preprocess(latestdata)
test_X

<ipython-input-25-6cd614beb98d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_time'] = pd.to_datetime(df['current_time'])
<ipython-input-25-6cd614beb98d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weekday'] = df['current_time'].apply(lambda x: x.strftime('%A'))
<ipython-input-25-6cd614beb98d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

array([[30.98, 16.5 ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  1.  ],
       [30.98, 16.5 ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  1.  ],
       [30.98, 16.5 ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  1.  ],
       [34.44, 16.5 ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  1.  ],
       [31.34, 16.5 ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  1.  ,  1.  ]])

In [28]:
timenow = datetime.datetime.now()
newdata = []
for _ in range(8):
    timenow = timenow + datetime.timedelta(minutes=15)
    predicted_y = model2.predict(test_X)
    break

TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'numpy.ndarray'>)